In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/ResnetTPU")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from tpu.models.official.resnet import resnet_preprocessing
from glob import glob
import math
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard
import datetime

CLASSES = 5
TRAIN_IMAGES=2934
VAL_IMAGES=364
TEST_IMAGES=372
EPOCHS = 15
BATCH_SIZE=256

In [0]:
def dataset_parser(value, is_training):
    """Parses an image and its label from a serialized ResNet-50 TFExample.

    Args:
      value: serialized string containing an ImageNet TFExample.

    Returns:
      Returns a tuple of (image, label) from the TFExample.
    """
    keys_to_features = {
        'image/encoded': tf.FixedLenFeature((), tf.string, ''),
        'image/class/label': tf.FixedLenFeature([], tf.int64, -1),
    }

    parsed = tf.parse_single_example(value, keys_to_features)
    image_bytes = tf.reshape(parsed['image/encoded'], shape=[])

    image = resnet_preprocessing.preprocess_image(
        image_bytes=image_bytes,
        is_training=is_training,
        image_size=299,
        use_bfloat16=False)

    # Subtract one so that labels are in [0, 1000).
    label = tf.cast(
        tf.reshape(parsed['image/class/label'], shape=[]), dtype=tf.int32) - 1

    label = tf.one_hot(label, CLASSES)

    return image, label


def create_dataset(dir, batch_size, is_training):
    dataset = tf.data.Dataset.list_files(file_pattern="tf-records/"+dir+"-*",shuffle=False)
      
    def fetch_dataset(filename):
      dataset = tf.data.TFRecordDataset(filenames = filename, buffer_size=1024*1024)
      return dataset
      
    # Read the data from disk in parallel
    dataset = dataset.apply(
        tf.data.experimental.parallel_interleave(map_func=fetch_dataset, cycle_length=4, sloppy=True))
    
#     dataset = dataset.repeat()
    
    dataset = dataset.cache().apply(
        tf.data.experimental.shuffle_and_repeat(buffer_size=batch_size*16))
    
    dataset = dataset.apply(
        tf.data.experimental.map_and_batch(
            lambda x : dataset_parser(x, is_training),
            batch_size=batch_size,
            num_parallel_calls=4,
            drop_remainder=True))

    dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
    return dataset

In [4]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(299,299,3), classes=CLASSES)

x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
   
# transfer learning
for layer in base_model.layers:
    layer.trainable = False

log_dir = "logs/"+datetime.datetime.now().strftime("%m-%d-%Y %H:%M:%S")
tensorboard = TensorBoard(log_dir=log_dir)
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(create_dataset("train", BATCH_SIZE, True), epochs=EPOCHS, steps_per_epoch=math.ceil(TRAIN_IMAGES/BATCH_SIZE), 
          validation_data=create_dataset("validation", BATCH_SIZE, False), validation_steps=math.ceil(VAL_IMAGES/BATCH_SIZE)
          , callbacks=[tensorboard])

model.save('flowers-gpu.h5')

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
12/12 [==============================] - 71s 6s/step - loss: 1.1947 - acc: 0.5394 - val_loss: 0.7286 - val_acc: 0.7109
Epoch 2/15
12/12 [==============================] - 50s 4s/step - loss: 0.5991 - acc: 0.7780 - val_loss: 0.5964 - val_acc: 0.7734
Epoch 3/15
12/12 [==============================] - 50s 4s/step - loss: 0.5136 - acc: 0.8203 - val_loss: 0.6136 - val_acc: 0.7754
Epoch 4/15
12/12 [==============================] - 50s 4s/step - loss: 0.4362 - acc: 0.8428 - val_loss: 0.5933 

In [6]:
from tensorflow.keras.models import load_model
import math
model = load_model('flowers-gpu.h5')
model.evaluate(create_dataset("test", BATCH_SIZE, False), steps=math.ceil(TEST_IMAGES/BATCH_SIZE))

2/2 [==============================] - 8s 4s/step - loss: 0.4743 - acc: 0.8223


[0.4742580056190491, 0.8222656]

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-03-25 19:16:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.86.186.182, 54.152.127.232, 52.87.35.92, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.86.186.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13584026 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.95M  26.4MB/s    in 0.5s    

2019-03-25 19:16:43 (26.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13584026/13584026]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
LOG_DIR = 'logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://e7f93fcd.ngrok.io
